In [ ]:
from jiant.utils.python import io as py_io

task_names = ["factuality", "kg_relations", "megaveridicality", "ner", "puns", "sentiment", "verbcorner", "verbnet", "winogender"]

def make_curriculum_dataset(task_name):
    train_data = py_io.read_jsonl(f"./curriculum/{task_name}/train.jsonl")
    test_data = py_io.read_jsonl(f"./curriculum/{task_name}/test.jsonl")

    train_examples = []
    test_examples = []

    for i, data in enumerate(train_data):
        if task_name == "kg_relations":
            label = data["binary-label"]
        else:
            #label = data["label"]
            update_type = data['UpdateType']
            if update_type == "strengthener":
                label = "entailed"
            else:
                label = "not-entailed"
        example = {
            "id": i,
            "premise": data["Update"],
            "hypothesis": data["Hypothesis"],
            "gold_label": label
        }
        train_examples.append(example)

    for i, data in enumerate(test_data):
        if task_name == "kg_relations":
            label = data["binary-label"]
        else:
            #label = data["label"]
            update_type = data['UpdateType']
            if update_type == "strengthener":
                label = "entailed"
            else:
                label = "not-entailed"
        example = {
            "id": i,
            "premise": data["Update"],
            "hypothesis": data["Hypothesis"],
            "gold_label": label
        }
        test_examples.append(example)

    os.makedirs("/content/tasks/configs/", exist_ok=True)
    os.makedirs(f"/content/tasks/curriculum/{task_name}", exist_ok=True)
    py_io.write_jsonl(
        data=train_examples, #train_darta,
        path=f"/content/tasks/curriculum/{task_name}/train.jsonl",
    )
    py_io.write_jsonl(
        data=test_examples,
        path=f"/content/tasks/curriculum/{task_name}/val.jsonl",
    )
    py_io.write_json({
    "task": f"{task_name}",
    "paths": {
        "train": f"/content/tasks/curriculum/{task_name}/train.jsonl",
        "val": f"/content/tasks/curriculum/{task_name}/val.jsonl",
    },
    "name": f"{task_name}"
    }, f"/content/tasks/configs/{task_name}_config.json")

#for task_name in task_names:
#    make_curriculum_dataset(task_name)
#    print(f"Build {task_name} dataset.")
make_curriculum_dataset("social_chem")

In [ ]:
import pandas as pd

train_df = pd.read_table('./curriculum/fava/combined/train.tsv')
test_df = pd.read_table('./curriculum/fava/combined/test.tsv')
dev_df = pd.read_table('./curriculum/fava/combined/dev.tsv')

train_df.head()

In [ ]:
premises = []
hypothesiss = []

count = 1
for row in test_df.itertuples(index=False):
  if count % 2 == 0:
    hypothesiss.append(row)
  else:
    premises.append(row)
  count += 1
  #premise = row.premise
  #sources = row.source.replace("[","").replace("]","").split("', ")
  #targets = row.target.replace("[","").replace("]","").split("', ")
  #relations = row.labels.replace("[","").replace("]","").split("', ")

In [ ]:
count = 1
for row in dev_df.itertuples(index=False):
  if count % 2 == 0:
    hypothesiss.append(row)
  else:
    premises.append(row)
  count += 1

In [ ]:
train_examples = []

for i in range(len(premises)):
  premise_row = premises[i]
  hypothesis_row = hypothesiss[i]

  premise = premise_row.sentence
  hypothesis = hypothesis_row.sentence
  lable = hypothesis_row.label
  if lable == 0:
    gold_label = "not-entailed"
  else:
    gold_label = "entailed"

  example = {
      "id": i,
      "verb_frame": premise_row.category,
      "premise": premise,
      "hypothesis": hypothesis,
      "gold_label": gold_label
  }
  train_examples.append(example)

py_io.write_jsonl(
    data=train_examples,
    path=f"/content/tasks/curriculum/syntactic_alternation/val.jsonl",
)

In [ ]:
task_name = "syntactic_alternation"

py_io.write_json({
    "task": f"{task_name}",
    "paths": {
        "train": f"/content/tasks/curriculum/{task_name}/train.jsonl",
        "val": f"/content/tasks/curriculum/{task_name}/val.jsonl",
    },
    "name": f"{task_name}"
    }, f"/content/tasks/configs/{task_name}_config.json")

In [ ]:
def model_test(model, test_data):
  premises = []
  hypothesis = []
  labels = []
  for data in test_data:
    premises.append(data["premise"])
    hypothesis.append(data["hypothesis"])
    labels.append(data["gold_label"])

  classes = ["entailment", "contradiction", "neutral"]
  classes = ["entailed", "not-entailed"]

  num_correct = 0
  for i in tqdm(range(len(premises))):
    test_sentence = tokenizer(premises[i], hypothesis[i], return_tensors="pt")
    test_sentence.to('cuda')
    logits = model(**test_sentence).logits
    out = torch.softmax(logits, dim=1)
    pred = torch.argmax(out).cpu().numpy()
    if pred == classes.index(labels[i]):
      num_correct += 1

  acc  = num_correct * 100 / len(premises)
  return acc

In [ ]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

bert_base = "textattack/bert-base-uncased-MNLI"
bert_large = "sentence-transformers/bert-large-nli-mean-tokens"
roberta_base = "textattack/roberta-base-MNLI"
roberta_large = "roberta-large-mnli"
deberta_base = "microsoft/deberta-base-mnli"
bart_large="textattack/facebook-bart-large-MNLI"

tokenizer = AutoTokenizer.from_pretrained(roberta_large)
model = AutoModelForSequenceClassification.from_pretrained(roberta_large)
model.cuda()

In [ ]:
task_names = ["factuality", "kg_relations", "megaveridicality", "ner", "puns", "sentiment", "verbcorner", "verbnet", "winogender"]
semanitc_nli_eval = {}

for task in task_names:
  test_data = py_io.read_jsonl(f"/content/tasks/curriculum/{task}/val.jsonl")
  acc = model_test(model, test_data)
  semanitc_nli_eval[task] = acc

py_io.write_json(semanitc_nli_eval, "./curriculum/Semantic/roberta-large-eval.json")

In [ ]:
semanitc_nli_eval

In [ ]:
py_io.write_json(semanitc_nli_eval, "./curriculum/semantic-roberta-large-eval.json")

In [ ]:
task_names = ["boolean", "comparative", "conditional", "counting", "negation", "quantifier", "monotonicity_simple", "monotonicity_hard"]
logical_nli_eval = {}

for task in task_names:
  test_data = py_io.read_jsonl(f"./curriculum/Logical/{task}/test/test.json")
  acc = model_test(model, test_data)
  logical_nli_eval[task] = acc

py_io.write_json(logical_nli_eval, "./curriculum/Logical/bert-large-eval.json")

In [ ]:
definitions = py_io.read_jsonl("./curriculum/World/dictionary_qa/train.jsonl")

In [ ]:
len(definitions)

In [ ]:
from nltk.corpus import wordnet as wn

def get_antonyms(input_lemma):
    antonyms = []
    for syn in wn.synsets(input_lemma):
        for lemma in syn.lemmas():
            if lemma.antonyms():
                antonyms.append(lemma.antonyms()[0].name())
    if len(antonyms) > 0:
        return antonyms[0]
    else:
        return None

In [ ]:
for syn in wn.synsets("separation"):
  print(syn.definition())

In [ ]:
import random

def build_definition_example(question):
  stem = question['question']['stem']
  stem_ls = stem.split("'")
  premise = stem_ls[1]
  surface_form = stem_ls[3]
  #antonym = get_antonyms(surface_form)
  answer_key = int(question['answerKey'])
  keys = [0,1,2,3,4]
  keys.remove(answer_key)
  wront_key = random.choice(keys)
  definition = question['question']['choices'][answer_key]['text']
  irrelevent = question['question']['choices'][wront_key]['text']
  hypothesis = premise.replace(surface_form, definition)
  neutral = premise.replace(surface_form, irrelevent)
  #if not antonym is None:
  #  contradict = premise.replace(surface_form, wn.synsets(antonym)[0].definition())
  #  return {"premise":premise, "hypothesis":hypothesis, "neutral":neutral, "contradict":contradict}
  return {"premise":premise, "hypothesis":hypothesis, "neutral":neutral}

In [ ]:
definition_qa_nli = []

i = 0
for index, question in enumerate(definitions):
  example = build_definition_example(question)
  entail = {"id":i, "sentence1": example["premise"], "sentence2": example["hypothesis"], "gold_label": "entailment"}
  definition_qa_nli.append(entail)
  i += 1
  neutral = {"id":i, "sentence1": example["premise"], "sentence2": example["neutral"], "gold_label": "neutral"}
  definition_qa_nli.append(neutral)
  i += 1
  #if len( example["contradict"]) > 0:
  #  contradict = {"id":i, "premise": example["premise"], "hypothesis": example["contradict"], "gold_label": "neutral"}
  #  definition_qa_nli.append(contradict)
  #  i += 1

In [ ]:
py_io.write_jsonl(definition_qa_nli, "./curriculum/dictionary_nli/train.jsonl")

# Read OntoNotes Data Copra

In [ ]:
from allennlp_models.common.ontonotes import Ontonotes, OntonotesSentence

ontonotes_train_dataset = Ontonotes()
train_iterator = ontonotes_train_dataset.dataset_iterator("./curriculum/wsj")

In [ ]:
train_instances = list(ontonotes_train_dataset.read("./curriculum/wsj"))

# Generate K-shot Data

In [ ]:
import argparse
import os
import numpy as np
import pandas as pd
from pandas import DataFrame
from jiant.utils.python.io import read_jsonl, write_jsonl

def load_curriculum_datasets(data_dir, tasks):
    datasets = {}
    for task in tasks:
        dataset = {}
        dirname = os.path.join(data_dir, task)
        splits = ["train", "val"]
        for split in splits:
            filename = os.path.join(dirname, f"{split}.jsonl")
            dataset[split] = read_jsonl(filename)
        datasets[task] = dataset
    return datasets

In [ ]:
args_dict = dict(
    k=32,
    task=["lexical_nli", "socialqa_nli"],
    seed=[100, 13, 21, 42, 87],
    data_dir="/content/tasks/data",
    output_dir="./few_shot/",
    mode='k-shot', # k-shot-10x
)

args = argparse.Namespace(**args_dict)
args.output_dir = os.path.join(args.output_dir, args.mode)

In [ ]:

k = args.k
print("K =", k)
datasets = load_curriculum_datasets(args.data_dir, args.task)

for seed in args.seed:
    print("Seed = %d" % (seed))
    for task, dataset in datasets.items():
        # Set random seed
        np.random.seed(seed)

        # Shuffle the training set
        print("| Task = %s" % (task))
        train_lines = dataset['train']
        np.random.shuffle(train_lines)

        # Set up dir
        task_dir = os.path.join(args.output_dir, task)
        setting_dir = os.path.join(task_dir, f"{k}-{seed}")
        os.makedirs(setting_dir, exist_ok=True)

        # Write test splits
        write_jsonl(dataset['val'], os.path.join(
            setting_dir, 'val.jsonl'))

        # Get label list for balanced sampling
        label_list = {}
        for line in train_lines:
            label = line['gold_label']
            if label not in label_list:
                label_list[label] = [line]
            else:
                label_list[label].append(line)

        new_train = []
        for label in label_list:
            new_train += label_list[label][:k]
        write_jsonl(new_train, os.path.join(
            setting_dir, 'train.jsonl'))

        new_dev = []
        for label in label_list:
            dev_rate = 11 if '10x' in args.mode else 2
            for line in label_list[label][k:k*dev_rate]:
                new_dev.append(line)
        write_jsonl(new_dev, os.path.join(
            setting_dir, 'dev.jsonl'))

In [1]:
from datetime import datetime
from typing import Optional

import datasets
import torch
import pytorch_lightning as pl

from torch.utils.data import DataLoader
from transformers import (
    AdamW,
    AutoConfig,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup,
)
from pytorch_lightning import LightningDataModule, LightningModule, Trainer

AVAIL_GPUS = min(1, torch.cuda.device_count())


class GLUEDataModule(LightningDataModule):

    glue_task_num_labels = {
        "lexical":3,
        "syntactic_alternation": 2,
        "monotonicity_infer": 2,
        "boolean": 3,
        "comparative": 3,
        "conditional": 3,
        "counting": 3,
        "negation": 3,
        "quantifier": 3,
        "monotonicity_simple": 3,
        "monotonicity_hard": 3,
        "factuality": 2,
        "kg_relations": 2,
        "megaveridicality": 2,
        "ner": 2,
        "puns": 2,
        "sentiment": 2,
        "verbcorner": 2,
        "verbnet": 2,
        "winogender: 2"
        "coreference": 2,
        "atomic": 2,
        "socail_chem": 2,
        "socialqa": 2,
        "physicalqa": 2,
        "context_align": 2,
    }

    loader_columns = [
        "datasets_idx",
        "input_ids",
        "token_type_ids",
        "attention_mask",
        "start_positions",
        "end_positions",
        "labels",
    ]

    def __init__(
        self,
        model_name_or_path: str,
        task_name: str = "atomic",
        max_seq_length: int = 128,
        train_batch_size: int = 32,
        eval_batch_size: int = 32,
        **kwargs,
    ):
        super().__init__()
        self.model_name_or_path = model_name_or_path
        self.task_name = task_name
        self.max_seq_length = max_seq_length
        self.train_batch_size = train_batch_size
        self.eval_batch_size = eval_batch_size

        self.text_fields = ['premise', 'hypothesis']
        self.num_labels = self.glue_task_num_labels[task_name]
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name_or_path, use_fast=True)

    def setup(self, stage: str):
        self.dataset = datasets.load_dataset("curriculum_load_dataset.py", self.task_name)

        for split in self.dataset.keys():
            self.dataset[split] = self.dataset[split].map(
                self.convert_to_features,
                batched=True,
                remove_columns=["label"],
            )
            self.columns = [c for c in self.dataset[split].column_names if c in self.loader_columns]
            self.dataset[split].set_format(type="torch", columns=self.columns)

        self.eval_splits = [x for x in self.dataset.keys() if "validation" in x]

    def prepare_data(self):
        datasets.load_dataset("curriculum_load_dataset.py", self.task_name)
        AutoTokenizer.from_pretrained(self.model_name_or_path, use_fast=True)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.train_batch_size)

    def val_dataloader(self):
        if len(self.eval_splits) == 1:
            return DataLoader(self.dataset["validation"], batch_size=self.eval_batch_size)
        elif len(self.eval_splits) > 1:
            return [DataLoader(self.dataset[x], batch_size=self.eval_batch_size) for x in self.eval_splits]

    def test_dataloader(self):
        if len(self.eval_splits) == 1:
            return DataLoader(self.dataset["test"], batch_size=self.eval_batch_size)
        elif len(self.eval_splits) > 1:
            return [DataLoader(self.dataset[x], batch_size=self.eval_batch_size) for x in self.eval_splits]

    def convert_to_features(self, example_batch, indices=None):

        # Either encode single sentence or sentence pairs
        if len(self.text_fields) > 1:
            texts_or_text_pairs = list(zip(example_batch[self.text_fields[0]], example_batch[self.text_fields[1]]))
        else:
            texts_or_text_pairs = example_batch[self.text_fields[0]]

        # Tokenize the text/text pairs
        features = self.tokenizer.batch_encode_plus(
            texts_or_text_pairs, max_length=self.max_seq_length, pad_to_max_length=True, truncation=True
        )

        # Rename label to labels to make it easier to pass to model forward
        features["labels"] = example_batch["label"]

        return features

In [2]:
class GLUETransformer(LightningModule):
    def __init__(
        self,
        model_name_or_path: str,
        num_labels: int,
        train_loader: DataLoader,
        task_name: str,
        learning_rate: float = 1e-5,
        adam_epsilon: float = 1e-8,
        warmup_steps: int = 0,
        weight_decay: float = 0.0,
        train_batch_size: int = 8,
        eval_batch_size: int = 16,
        eval_splits: Optional[list] = None,
        **kwargs,
    ):
        super().__init__()

        self.save_hyperparameters()
        self.train_loader = train_loader
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
        self.config = AutoConfig.from_pretrained(model_name_or_path, num_labels=num_labels)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, config=self.config)
        self.metric = datasets.load_metric(
            "glue", 'mnli', experiment_id=datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
        )

    def forward(self, **inputs):
        return self.model(**inputs)

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs[0]
        return loss

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        outputs = self(**batch)
        val_loss, logits = outputs[:2]

        if self.hparams.num_labels >= 1:
            preds = torch.argmax(logits, axis=1)
        elif self.hparams.num_labels == 1:
            preds = logits.squeeze()

        labels = batch["labels"]

        return {"loss": val_loss, "preds": preds, "labels": labels}

    def validation_epoch_end(self, outputs):
        preds = torch.cat([x["preds"] for x in outputs]).detach().cpu().numpy()
        labels = torch.cat([x["labels"] for x in outputs]).detach().cpu().numpy()
        loss = torch.stack([x["loss"] for x in outputs]).mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log_dict(self.metric.compute(predictions=preds, references=labels), prog_bar=True)
        return loss

    def setup(self, stage=None) -> None:
        if stage != "fit":
            return

        # Calculate total steps
        tb_size = self.hparams.train_batch_size * max(1, self.trainer.gpus)
        ab_size = self.trainer.accumulate_grad_batches * float(self.trainer.max_epochs)
        self.total_steps = (len(self.train_loader.dataset) // tb_size) // ab_size

    def configure_optimizers(self):
        """Prepare optimizer and schedule (linear warmup and decay)"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.hparams.warmup_steps,
            num_training_steps=self.total_steps,
        )
        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}
        return [optimizer], [scheduler]

In [6]:
from pytorch_lightning.loggers import TensorBoardLogger

dm = GLUEDataModule(
    model_name_or_path="roberta-base",
    task_name="context_align",
)
dm.setup("fit")

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=f"./runs/{dm.task_name}/roberta-base/checkpoints/",
    filename="checkpoint_best",
    monitor="val_loss",
    mode="min",
    save_top_k=2
)

train_params = dict(
    gpus=1,
    max_epochs=5,
    progress_bar_refresh_rate=1,
    checkpoint_callback=True,
    callbacks=[checkpoint_callback],
    logger=TensorBoardLogger(
        os.path.join("./runs", 'logs'),
        name=f"{dm.model_name_or_path}-{dm.task_name}",
        version='trial_1')
)

Dataset curriculum downloaded and prepared to C:\Users\Admin\.cache\huggingface\datasets\curriculum\context_align\1.0.0\546cd813c08ed2b2471465ee3c2679b6e9d71927c250a97b6f8833e072fc38e0. Subsequent calls will reuse this data.


  0%|          | 0/15 [00:00<?, ?ba/s]C:\Users\Admin\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 2/2 [00:00<00:00, 13.29ba/s]


In [7]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

model = GLUETransformer(
    model_name_or_path="roberta-base",
    num_labels=dm.num_labels,
    eval_splits=dm.eval_splits,
    task_name=dm.task_name,
    train_loader=dm.train_dataloader()
)

trainer = Trainer(**train_params)
trainer.fit(model, dm)
trainer.validate(model, dm.val_dataloader())

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\Admin\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


C:\Users\Admin\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 4: 100%|██████████| 494/494 [02:12<00:00,  3.74it/s, loss=2.7, v_num=al_1, val_loss=1.190, accuracy=0.604]


Reusing dataset curriculum (C:\Users\Admin\.cache\huggingface\datasets\curriculum\context_align\1.0.0\546cd813c08ed2b2471465ee3c2679b6e9d71927c250a97b6f8833e072fc38e0)
100%|██████████| 2/2 [00:00<00:00, 10.87ba/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'accuracy': 0.6036446690559387, 'val_loss': 1.1908454895019531}
--------------------------------------------------------------------------------


[{'val_loss': 1.1908454895019531, 'accuracy': 0.6036446690559387}]

In [8]:
validate_model = GLUETransformer.load_from_checkpoint("./runs/context_align/roberta-base/checkpoints/checkpoint_best.ckpt")
trainer = Trainer(**train_params)
trainer.validate(validate_model, dm.val_dataloader())

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'accuracy': 0.6036446690559387, 'val_loss': 1.1908454895019531}
--------------------------------------------------------------------------------


[{'val_loss': 1.1908454895019531, 'accuracy': 0.6036446690559387}]

In [6]:
import jiant.scripts.download_data.runscript as downloader

DATA_DIR = "/content/tasks"

downloader.download_data(["adversarial_nli_r1"], f"{DATA_DIR}")
downloader.download_data(["adversarial_nli_r2"], f"{DATA_DIR}")
downloader.download_data(["adversarial_nli_r3"], f"{DATA_DIR}")

Using custom data configuration default
Reusing dataset anli (C:\Users\Admin\.cache\huggingface\datasets\anli\default\None\aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b)


Downloaded and generated configs for 'adversarial_nli_r1' (1/1)


Using custom data configuration default
Reusing dataset anli (C:\Users\Admin\.cache\huggingface\datasets\anli\default\None\aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b)


Downloaded and generated configs for 'adversarial_nli_r2' (1/1)


Using custom data configuration default
Reusing dataset anli (C:\Users\Admin\.cache\huggingface\datasets\anli\default\None\aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b)


Downloaded and generated configs for 'adversarial_nli_r3' (1/1)


In [9]:
downloader.download_data(["mrpc"], f"{DATA_DIR}")

Reusing dataset glue (C:\Users\Admin\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Downloaded and generated configs for 'mrpc' (1/1)
